In [1]:
import os
import pandas as pd
import datetime
import re
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def is_expected_group_format(group_name, team_name):
    if team_name.lower() == 'sales':
        pattern = re.compile(r'.+?_(EDOOFA|edoofa|Edoofa|EA)|\(\d{2}_\d{2}\)')
        return bool(pattern.search(group_name))
    else:
        parts = group_name.split()
        return len(parts) >= 2 and not (re.search(r'[^A-Za-z0-9]', parts[0]) or re.search(r'[^A-Za-z0-9]', parts[1]))

def parse_chat_group_name(file_path):
    team_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
    file_name = os.path.basename(file_path)
    
    # Remove the "WhatsApp Chat with " prefix and the file extension
    group_name_with_extension = file_name.replace("WhatsApp Chat with ", "")
    group_name = os.path.splitext(group_name_with_extension)[0]
    
    # Check for duplicates like (1), (2) etc., and remove them
    group_name = re.sub(r'\(\d+\)$', '', group_name)

    # Check if the group name is in the expected format
    expected_format = is_expected_group_format(group_name, team_name)
    return group_name, expected_format

def parse_chat_file_for_delay_analysis(file_path):
    person_name = os.path.basename(os.path.dirname(file_path))
    folder_date_str = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_path))))
    expected_date_minus_one = pd.to_datetime(folder_date_str, format='%Y-%m-%d').date() - datetime.timedelta(days=1)

    delay_messages = []

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    last_person_msg_time = None

    for i, line in enumerate(lines):
        message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
        if message_match:
            date_time_str, sender, message = message_match.groups()
            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and not (sender.strip().isnumeric() or sender.startswith('+'))

            if is_person:
                if last_person_msg_time is not None:
                    time_diff_seconds = (date_time - last_person_msg_time).total_seconds()
                    time_diff_minutes = time_diff_seconds / 60
                    delay = time_diff_minutes > 15
                else:
                    delay = False
                last_person_msg_time = date_time
            else:
                delay = False

            if delay:
                delay_messages.append((date_time, sender, message))

    return delay_messages

def process_all_files(chat_files):
    all_delay_messages = []

    for file_path in chat_files:
        logging.info(f"Processing file: {file_path}")  # Debug line
        delay_messages = parse_chat_file_for_delay_analysis(file_path)

        for message in delay_messages:
            timestamp, sender, msg = message
            chat_group_name, expected_format = parse_chat_group_name(file_path)
            if expected_format:
                all_delay_messages.append((timestamp, sender, chat_group_name, True))

    return all_delay_messages

def create_delay_data_dataframe(all_delay_data):
    columns = ['Date', 'Person', 'Chat Group Name', 'Delay']
    df_data = []

    for delay_instance in all_delay_data:
        date_time, sender, chat_group_name, delay = delay_instance
        df_data.append([date_time, sender, chat_group_name, delay])

    delay_df = pd.DataFrame(df_data, columns=columns)
    return delay_df

# Main Execution
date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231130T060206Z-001"
chat_files = list_chat_files(date_directory)
all_delay_data = process_all_files(chat_files)
delay_analysis_df = create_delay_data_dataframe(all_delay_data)

# Display the first few rows of the DataFrame
print(delay_analysis_df.head())


2023-11-30 17:35:37,741 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Abdul EWYL23E0722.txt
2023-11-30 17:35:37,782 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Admire EWYL22E1163.txt
2023-11-30 17:35:37,794 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Allen EWYL22E0762.txt
2023-11-30 17:35:37,860 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Anesu EWYL22E0418.txt
2023-11-30 17:35:37,908 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder f

2023-11-30 17:35:38,887 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with LISA EWYL22E0237.txt
2023-11-30 17:35:38,932 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Lorenzo EWYL22E0908.txt
2023-11-30 17:35:38,938 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Mbalenhle EWYL23E0069.txt
2023-11-30 17:35:38,978 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Melisa EWYL23E0145.txt
2023-11-30 17:35:38,988 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Fol

2023-11-30 17:35:40,015 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Shammah EWYL22E1246.txt
2023-11-30 17:35:40,086 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Sharon EWYL22E0561.txt
2023-11-30 17:35:40,125 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Shelter EWYL23E0304.txt
2023-11-30 17:35:40,213 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Shingirai EWYL23E0606.txt
2023-11-30 17:35:40,242 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat 

2023-11-30 17:35:41,881 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Amani EWYL23E0532.txt
2023-11-30 17:35:41,958 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Anesuishe EWYL22E1011.txt
2023-11-30 17:35:42,007 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Anopa EWYL23E0772.txt
2023-11-30 17:35:42,027 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Arnold EWYL22E1036.txt
2023-11-30 17:35:42,067 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat 

2023-11-30 17:35:43,411 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kelly EWYL22E0137.txt
2023-11-30 17:35:43,437 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kelvin EWYL22E0590.txt
2023-11-30 17:35:43,537 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kudakwashe EWYL23E0289.txt
2023-11-30 17:35:43,577 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kudzaiishe EWYL22E1214.txt
2023-11-30 17:35:43,634 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2

...

2023-11-30 17:36:24,971 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Misheck Mutunami Edoofa (18_11) SZ.txt
2023-11-30 17:36:24,983 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Mishel Tadiwa Diwura Edoofa (16_11) SZ.txt
2023-11-30 17:36:25,000 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Mitchell Mbiri Edoofa (28_11) SZ.txt
2023-11-30 17:36:25,003 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Mukangaya Kudzai C Edoofa (09_10) SZ.txt
2023-11-30 1

2023-11-30 17:36:25,400 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tariro Chiwara Edoofa (21_11) SZ.txt
2023-11-30 17:36:25,440 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tarutira Anopaishe Edoofa (28_11) SZ.txt
2023-11-30 17:36:25,444 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tashinga Muvirimi Edoofa (29_11) SZ.txt
2023-11-30 17:36:25,448 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tawanda M Muzandaka Edoofa (14_11) SZ.txt
2023-11-30 1

2023-11-30 17:36:26,470 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kudakwashe_EdoofaAFFCA(03_10)SM.txt
2023-11-30 17:36:26,538 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kudzai Ndongwe_EDOOFA(20_11)SM23.txt
2023-11-30 17:36:26,569 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kunashe Sai_Edoofa(10_10)SM.txt
2023-11-30 17:36:26,633 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kundai Nyamhanza_EDOOFA(14_11)SM23.txt
2023-11-30 17:36:26,652 - INFO - Pro

2023-11-30 17:36:27,774 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Shingirai Vunganai_EA(16_11)TR.txt
2023-11-30 17:36:27,826 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Sigireta Tariro_EDOOFA(22_11)GS.txt
2023-11-30 17:36:27,862 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Simeon Wamedzwa_EDOOFA(16_11)SM23.txt
2023-11-30 17:36:27,879 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Solomon Chere_EDOOFAMHS(14_11)SM23.txt
2023-11-30 17:36:27,942 - INFO -

                 Date        Person     Chat Group Name  Delay
0 2023-11-29 14:23:00  Aditi Edoofa  Admire EWYL22E1163   True
1 2023-11-29 13:51:00  Aditi Edoofa   Allen EWYL22E0762   True
2 2023-11-29 15:19:00  Aditi Edoofa   Allen EWYL22E0762   True
3 2023-11-29 14:52:00  Aditi Edoofa   Anesu EWYL22E0418   True
4 2023-11-29 15:17:00  Aditi Edoofa   Anesu EWYL22E0418   True


# Expected Format Checks

In [ ]:
delay_analysis_df.to_csv("delay_1.csv",index=False)

In [2]:
import os
import pandas as pd
import datetime
import re
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def is_expected_group_format(group_name, team_name):
    if team_name.lower() == 'sales':
        pattern = re.compile(r'.+?_(EDOOFA|edoofa|Edoofa|EA)|\(\d{2}_\d{2}\)')
        return bool(pattern.search(group_name))
    else:
        parts = group_name.split()
        return len(parts) >= 2 and not (re.search(r'[^A-Za-z0-9]', parts[0]) or re.search(r'[^A-Za-z0-9]', parts[1]))

def parse_chat_group_name(file_path):
    team_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
    file_name = os.path.basename(file_path)
    
    # Remove the "WhatsApp Chat with " prefix and the file extension
    group_name_with_extension = file_name.replace("WhatsApp Chat with ", "")
    group_name = os.path.splitext(group_name_with_extension)[0]
    
    # Check for duplicates like (1), (2) etc., and remove them
    group_name = re.sub(r'\(\d+\)$', '', group_name)

    # Check if the group name is in the expected format
    expected_format = is_expected_group_format(group_name, team_name)
    return group_name, expected_format

def parse_chat_file_for_delay_analysis(file_path):
    person_name = os.path.basename(os.path.dirname(file_path))
    folder_date_str = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_path))))
    expected_date_minus_one = pd.to_datetime(folder_date_str, format='%Y-%m-%d').date() - datetime.timedelta(days=1)

    delay_messages = []

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    last_person_msg_time = None

    for i, line in enumerate(lines):
        message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
        if message_match:
            date_time_str, sender, message = message_match.groups()
            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and not (sender.strip().isnumeric() or sender.startswith('+'))

            if is_person:
                if last_person_msg_time is not None:
                    time_diff_seconds = (date_time - last_person_msg_time).total_seconds()
                    time_diff_minutes = time_diff_seconds / 60
                    delay = time_diff_minutes > 15
                else:
                    delay = False
                last_person_msg_time = date_time
            else:
                delay = False

            if delay:
                delay_messages.append((date_time, sender, message))

    return delay_messages

def process_all_files(chat_files):
    all_delay_messages = []

    for file_path in chat_files:
        logging.info(f"Processing file: {file_path}")  # Debug line
        delay_messages = parse_chat_file_for_delay_analysis(file_path)

        for message in delay_messages:
            timestamp, sender, msg = message
            chat_group_name, expected_format = parse_chat_group_name(file_path)
            all_delay_messages.append((timestamp, sender, chat_group_name, expected_format, True))

    return all_delay_messages

def create_delay_data_dataframe(all_delay_data):
    columns = ['Date', 'Person', 'Chat Group Name', 'Expected Format', 'Delay']
    df_data = []

    for delay_instance in all_delay_data:
        date_time, sender, chat_group_name, expected_format, delay = delay_instance
        df_data.append([date_time, sender, chat_group_name, expected_format, delay])

    delay_df = pd.DataFrame(df_data, columns=columns)
    return delay_df

# Main Execution
date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231130T060206Z-001"
chat_files = list_chat_files(date_directory)
all_delay_data = process_all_files(chat_files)
delay_analysis_df = create_delay_data_dataframe(all_delay_data)

# Display the first few rows of the DataFrame
print(delay_analysis_df.head())


2023-11-30 17:38:40,916 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Abdul EWYL23E0722.txt
2023-11-30 17:38:40,953 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Admire EWYL22E1163.txt
2023-11-30 17:38:40,964 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Allen EWYL22E0762.txt
2023-11-30 17:38:41,023 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Anesu EWYL22E0418.txt
2023-11-30 17:38:41,067 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder f

2023-11-30 17:38:42,001 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with LISA EWYL22E0237.txt
2023-11-30 17:38:42,047 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Lorenzo EWYL22E0908.txt
2023-11-30 17:38:42,053 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Mbalenhle EWYL23E0069.txt
2023-11-30 17:38:42,096 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Melisa EWYL23E0145.txt
2023-11-30 17:38:42,109 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Fol

2023-11-30 17:38:43,149 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Shammah EWYL22E1246.txt
2023-11-30 17:38:43,270 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Sharon EWYL22E0561.txt
2023-11-30 17:38:43,299 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Shelter EWYL23E0304.txt
2023-11-30 17:38:43,377 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Aditi_Edoofa\WhatsApp Chat with Shingirai EWYL23E0606.txt
2023-11-30 17:38:43,397 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat 

2023-11-30 17:38:44,625 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Amani EWYL23E0532.txt
2023-11-30 17:38:44,683 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Anesuishe EWYL22E1011.txt
2023-11-30 17:38:44,712 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Anopa EWYL23E0772.txt
2023-11-30 17:38:44,723 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Arnold EWYL22E1036.txt
2023-11-30 17:38:44,750 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat 

2023-11-30 17:38:45,658 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kelly EWYL22E0137.txt
2023-11-30 17:38:45,674 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kelvin EWYL22E0590.txt
2023-11-30 17:38:45,729 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kudakwashe EWYL23E0289.txt
2023-11-30 17:38:45,754 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\EWYL\Ananya_Edoofa\WhatsApp Chat with Kudzaiishe EWYL22E1214.txt
2023-11-30 17:38:45,787 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2

...

2023-11-30 17:44:19,399 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Misheck Mutunami Edoofa (18_11) SZ.txt
2023-11-30 17:44:19,424 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Mishel Tadiwa Diwura Edoofa (16_11) SZ.txt
2023-11-30 17:44:19,450 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Mitchell Mbiri Edoofa (28_11) SZ.txt
2023-11-30 17:44:19,454 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Mukangaya Kudzai C Edoofa (09_10) SZ.txt
2023-11-30 1

2023-11-30 17:44:20,064 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tariro Chiwara Edoofa (21_11) SZ.txt
2023-11-30 17:44:20,131 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tarutira Anopaishe Edoofa (28_11) SZ.txt
2023-11-30 17:44:20,134 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tashinga Muvirimi Edoofa (29_11) SZ.txt
2023-11-30 17:44:20,134 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shashwat Edoofa 2\WhatsApp Chat with Tawanda M Muzandaka Edoofa (14_11) SZ.txt
2023-11-30 1

2023-11-30 17:44:21,419 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kudakwashe_EdoofaAFFCA(03_10)SM.txt
2023-11-30 17:44:21,515 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kudzai Ndongwe_EDOOFA(20_11)SM23.txt
2023-11-30 17:44:21,566 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kunashe Sai_Edoofa(10_10)SM.txt
2023-11-30 17:44:21,639 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Kundai Nyamhanza_EDOOFA(14_11)SM23.txt
2023-11-30 17:44:21,665 - INFO - Pro

2023-11-30 17:44:22,784 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Shingirai Vunganai_EA(16_11)TR.txt
2023-11-30 17:44:22,827 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Sigireta Tariro_EDOOFA(22_11)GS.txt
2023-11-30 17:44:22,861 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Simeon Wamedzwa_EDOOFA(16_11)SM23.txt
2023-11-30 17:44:22,874 - INFO - Processing file: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231130T060206Z-001\2023-11-30\SALES\Shubham Madhwal\WhatsApp Chat with Solomon Chere_EDOOFAMHS(14_11)SM23.txt
2023-11-30 17:44:22,934 - INFO -

                 Date        Person     Chat Group Name  Expected Format  \
0 2023-11-29 14:23:00  Aditi Edoofa  Admire EWYL22E1163             True   
1 2023-11-29 13:51:00  Aditi Edoofa   Allen EWYL22E0762             True   
2 2023-11-29 15:19:00  Aditi Edoofa   Allen EWYL22E0762             True   
3 2023-11-29 14:52:00  Aditi Edoofa   Anesu EWYL22E0418             True   
4 2023-11-29 15:17:00  Aditi Edoofa   Anesu EWYL22E0418             True   

   Delay  
0   True  
1   True  
2   True  
3   True  
4   True  


In [ ]:
delay_analysis_df.to_csv("delay_2.csv",index=False)